# 3) Vaja: NodeJS, Redis, docker-compose

Naredimo mapo `visits`

- Naredimo package.json in index.js

```json
{
  "dependencies": {
    "express": "*",
    "redis": "2.8.0"
  },
  "scripts": {
    "start": "node index.js"
  }
}


```

```javascript
const express = require("express");
const redis = require("redis");
const process = require("process");

const app = express();
const client = redis.createClient({
  host: "redis-server",
  port: 6379
});
client.set("visits", 0);

app.get("/", (req, res) => {
  //process.exit(0)
  client.get("visits", (err, visits) => {
    res.send("Number of visits " + visits);
    client.set("visits", parseInt(visits) + 1);
  });
});

app.listen(8081, () => {
  console.log("listening on port 8081");
});
```

Naredimo Dockerfile:

```Dockerfile
FROM node:alpine

WORKDIR '/app'

COPY package.json .
RUN npm install
COPY . .

CMD ["npm","start"]
```

Zgradimo sliko:
    
    docker build -t leon11sj/visits . 

Pokažemo na roko kako lahko zaženemo 2 kontejnerja.

    docker run redis
    docker run leon11sj/visits
    
Morali bi ročno narediti omrežje, kar ni najboljša praksa.

Boljša praksa pri več storitvah je uporabiti docker-compose.

```yml
version: "3"
services:
  redis-server:
    image: "redis"
  node-app:
    restart: on-failure
    build: .
    ports:
      - "4001:8081"
```

    docker-compose up
    
Rebuild the image

    docker-compose up --build 

Pokažemo da compose nardi novi network, da združi naše kontejnerje skupaj

Zaženemo kontejnerje v ozadju:
    
    docker-compose up -d 

Ustavimo:
    
    docker-compose down

Pokažemo kako kontejner pade, in kako ga lahko resetiramo
- odkomentiramo //process.exit(0)
- dodamao v docker-compose  restart: on-failure
- pokažemo različne stop kode (npr, 1,2)

<table>
  <thead>
    <tr>
      <th style="text-align: left">Flag</th>
      <th style="text-align: left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: left"><code class="highlighter-rouge">no</code></td>
      <td style="text-align: left">Do not automatically restart the container. (the default)</td>
    </tr>
    <tr>
      <td style="text-align: left"><code class="highlighter-rouge">on-failure</code></td>
      <td style="text-align: left">Restart the container if it exits due to an error, which manifests as a non-zero exit code.</td>
    </tr>
    <tr>
      <td style="text-align: left"><code class="highlighter-rouge">always</code></td>
      <td style="text-align: left">Always restart the container if it stops. If it is manually stopped, it is restarted only when Docker daemon restarts or the container itself is manually restarted. (See the second bullet listed in <a href="#restart-policy-details">restart policy details</a>)</td>
    </tr>
    <tr>
      <td style="text-align: left"><code class="highlighter-rouge">unless-stopped</code></td>
      <td style="text-align: left">Similar to <code class="highlighter-rouge">always</code>, except that when the container is stopped (manually or otherwise), it is not restarted even after Docker daemon restarts.</td>
    </tr>
  </tbody>
</table>

Pokaže samo kontejnerje, ki so zraven datoteke katero zaganjamo

    docker-compose ps